In [ ]:
# Download Essential libraries
!pip install flair
!pip install flask-ngrok
!pip install flask-bootstrap
!pip install pyngrok
!pip install yfinance
!pip install -U kaleido
!pip install pyyaml==5.4.1
!pip install wordcloud==1.8.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 401 kB 8.1 MB/s 
     |████████████████████████████████| 981 kB 41.1 MB/s 
     |████████████████████████████████| 1.2 MB 37.6 MB/s 
     |████████████████████████████████| 788 kB 47.9 MB/s 
     |████████████████████████████████| 46 kB 4.3 MB/s 
     |████████████████████████████████| 53 kB 1.7 MB/s 
     |████████████████████████████████| 19.7 MB 1.3 MB/s 
     |████████████████████████████████| 101 kB 10.7 MB/s 
     |████████████████████████████████| 1.6 MB 36.8 MB/s 
     |████████████████████████████████| 4.7 MB 46.5 MB/s 
     |████████████████████████████████| 199 kB 42.0 MB/s 
     |████████████████████████████████| 62 kB 1.3 MB/s 
     |████████████████████████████████| 596 kB 38.7 MB/s 
     |████████████████████████████████| 6.6 MB 40.7 MB/s 
  Created wheel for mpld3: filename=mpld3-0.3-py3-none-any.whl size=116702 sha256=18714c4ae596a7

In [ ]:
!pip uninstall decouple
!pip install python-decouple

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
# Import Libraries
import flair
sentiment_model = flair.models.TextClassifier.load('en-sentiment')
import pytz
tzone = pytz.timezone("utc")
import requests
import pandas as pd
from flask import (
    Flask,
    render_template,
    request,
    redirect,
    flash,
    url_for,
    current_app
)
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, request
import os
from google.colab import drive
import re
from datetime import datetime, timedelta
import requests
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import sys

2022-08-11 08:30:03,761 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-distilbert/sentiment-en-mix-distillbert_4.pt not found in cache, downloading to /tmp/tmpulc3_9do


100%|██████████| 265512723/265512723 [00:16<00:00, 16590385.78B/s]

2022-08-11 08:30:20,293 copying /tmp/tmpulc3_9do to cache at /root/.flair/models/sentiment-en-mix-distillbert_4.pt


2022-08-11 08:30:20,817 removing temp file /tmp/tmpulc3_9do
2022-08-11 08:30:20,855 loading file /root/.flair/models/sentiment-en-mix-distillbert_4.pt


/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [ ]:
# Drive folder access
drive.mount('/content/gdrive', force_remount=True)
sys.path.insert(0,'/content/gdrive/MyDrive/news_api-main')
TEMPLATE = '/content/gdrive/MyDrive/news_api-main/myflask/templates'
STATIC = '/content/gdrive/MyDrive/news_api-main/myflask/static'

Mounted at /content/gdrive


In [ ]:
import nltk

In [ ]:
from myflask import main_app

In [ ]:
# Authentication
!ngrok authtoken 
BEARER_TOKEN = ''

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
# Variables
tickers = {'tsla': 'tesla', 'msft': 'microsoft', 'amzn': 'amazon', 'ttm': 'tata motors', 'goog': 'google', 'infy': 'infosys'}

In [ ]:
# Tweet cleanse
def get_data(tweet):
    data = {
        'id': tweet['id'],
        'created_at': tweet['created_at'],
        'text': tweet['text']
    }
    return data

def clean(tweet, ticker):
  whitespace = re.compile(r"\s+")
  text = 'hello'
  web_address = re.compile(r"(?i)http(s):\/\/[a-z0-9.~_\-\/]+")
  tesla = re.compile(fr"(?i)@{ticker.title()}(?=\b)")
  user = re.compile(r"(?i)@[a-z0-9_]+")

  # we then use the sub method to replace anything matching
  tweet = whitespace.sub(' ', tweet)
  tweet = web_address.sub('', tweet)
  tweet = tesla.sub(f'{ticker.title()}', tweet)
  tweet = user.sub('', tweet)
  return tweet

# Get tweets
# we use this function to subtract 60 mins from our datetime string
def time_travel(now, mins):
    # the date format string required by twitter
    dtformat = '%Y-%m-%dT%H:%M:%SZ'
    now = datetime.strptime(now, dtformat)
    back_in_time = now - timedelta(minutes=mins)
    return back_in_time.strftime(dtformat)

# setup the API request
def query(BEARER_TOKEN, ticker, df):
  name = tickers[ticker]
  endpoint = 'https://api.twitter.com/2/tweets/search/recent'
  headers = {'authorization': f'Bearer {BEARER_TOKEN}'}
  params = {
      'query': f'({name} OR {ticker}) -is:retweet -is:reply (lang:en)',
      'max_results': '10',
      'tweet.fields': 'created_at,lang'
  }
  # the date format string required by twitter
  dtformat = '%Y-%m-%dT%H:%M:%SZ'  

  now = datetime.now()
  # get the current datetime, this is our starting point
  now = now - timedelta(days=0) 
  # datetime one week ago = the finish line
  last_week = now - timedelta(days=1)  
  now = now.strftime(dtformat)  # convert now datetime to format for API

  # df = pd.DataFrame()  # initialize dataframe to store tweets
  i=0
  pre60 = time_travel(now, 60)  # get 60 minutes before 'now'
  # assign from and to datetime parameters for the API
  params['start_time'] = pre60
  params['end_time'] = now
  # count = 0
  while True:
      if datetime.strptime(now, dtformat) < last_week:
          # if we have reached 7 days ago, break the loop
          break
      pre60 = time_travel(now, 60)  # get 60 minutes before 'now'
      # assign from and to datetime parameters for the API
      params['start_time'] = pre60
      params['end_time'] = now
      response = requests.get(endpoint,
                              params=params,
                              headers=headers)  # send the request
      now = pre60  # move the window 60 minutes earlier
      # iteratively append our tweet data to our dataframe
      # print(response)
      if (response.status_code==200):
        for tweet in response.json()['data']:
            row = get_data(tweet)  # we defined this function earlier
            if (df.empty):
              df = df.append(row, ignore_index=True)
            elif (len(df[row['text']==df['text']])==0):

            # print("-------------Tweet start--------------")
            # print(row)
            # print("---------------Tweet end--------------")
              df = df.append(row, ignore_index=True)
      else:
        print(response.status_code)
        if (response.status_code==429):
          break
      # count+=1
  return df

# Add sentiment
def add_sentiments(df, ticker):
  probs = []
  sentiments = []

  # use regex expressions (in clean function) to clean tweets
  df_text = []
  for tweet in df['text'].to_list():
    df_text.append(clean(tweet, ticker))
  
  df['text'] = df_text

  for tweet in df['text'].to_list():
      # make prediction
      sentence = flair.data.Sentence(tweet)
      sentiment_model.predict(sentence)
      # extract sentiment prediction
      probs.append(sentence.labels[0].score)  # numerical score 0-1
      sentiments.append(sentence.labels[0].value)  # 'POSITIVE' or 'NEGATIVE'

  # add probability and sentiment predictions to tweets dataframe
  df['probability'] = probs
  df['sentiment'] = sentiments
  df['polarity'] = df['sentiment'].apply(lambda x: 1 if x=='POSITIVE' else -1)
  df['total'] = df['probability']*df['polarity']
  return df

# Refine dataframe
def add_tzone(tzone, df):
  df['new_created_at'] = df['created_at'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ'))
  df['America/New_York Timezone'] = df['new_created_at'].apply(lambda x: tzone.localize(x))
  df['America/New_York Timezone'] = df['America/New_York Timezone'].apply(lambda x: x.tz_convert("America/New_York"))
  return df

def refine_df(tzone, df, ticker):
  df = add_sentiments(df, ticker)
  df = add_tzone(tzone, df)
  return df

# Yahooo Finance stock data
def get_stock_data(ticker, df):
  tick = yf.Ticker(f"{ticker.upper()}")
  stock_data = tick.history(
      start=(df['new_created_at'].min()).strftime('%Y-%m-%d'),
      end=(df['new_created_at'].max()).strftime('%Y-%m-%d'),
      interval='60m'
  ).reset_index()
  return stock_data

def generate_and_plot(stock_data, mean_sentiments, ticker):
  # plt.ylabel('Sentiment score')
  # plt.xlabel('Datetime')
  # plt.plot(stock_data['Datetime'], mean_sentiments)
  # plt.savefig(f"{STATIC}/{ticker}1.png")

  # plt.xlabel('Datetime')
  # plt.ylabel('Closing Price')
  # plt.plot(stock_data['Datetime'], stock_data['Close'])
  # plt.autoscale()
  # plt.savefig(f"{STATIC}/{ticker}.png")

  trace1 = go.Scatter(
      x=stock_data['Datetime'],
      y=stock_data['Close'],
      name='Closing Price',
  )

  trace2 = go.Scatter(
      x=stock_data['Datetime'],
      y=stock_data['Mean Sentiment'],
      name='Superimposed Average Sentiment 2 hours before'
  )

  # trace3 = go.Scatter(
  #     x=stock_data['Datetime'],
  #     y=stock_data['Mean Sentiment'],
  #     name='Imposed Sentiment Score'
  # )

  fig = make_subplots()
  fig.add_trace(trace1)
  fig.add_trace(trace2)
  fig.write_image(f"{STATIC}/images/{ticker}_tweet.png")

  # fig = px.line(stock_data, x="Datetime", y="Close", title=f'{ticker.title()} Closing Price and Sentiments')
  # fig = px.line(stock_data, x="Datetime", y="Mean Sentiment")
  # fig.show()

# Generate results
def make_data(ticker, df):
  stock_data = get_stock_data(ticker, df)
  # print(stock_data.head())
  mean_closing = stock_data['Close'].mean()
  max_closing = stock_data['Close'].max()
  min_closing = stock_data['Close'].min()
  range_closing = (max_closing-min_closing)/2
  print("Range: ", range_closing)
  # before930 = df[(df['America/New_York Timezone']>=stock_data.iloc[0]['Datetime']) & (df['America/New_York Timezone']<=stock_data.iloc[1]['Datetime'])]
  # normal_sentiments = [mean_closing]
  mean_sentiments = [mean_closing, mean_closing]
  for i in range(len(stock_data)-2):
    before = df[(df['America/New_York Timezone']>=stock_data.iloc[i]['Datetime']) & (df['America/New_York Timezone']<=stock_data.iloc[i+2]['Datetime'])]
    m = before['total'].mean()
    if (before.empty):
      m = mean_closing
    else:
      m = m*range_closing + mean_closing
      print(m, range_closing, mean_closing)
    mean_sentiments.append(m)
  stock_data['Mean Sentiment'] = mean_sentiments
  print("Mean sentiments", mean_sentiments)
  generate_and_plot(stock_data, mean_sentiments, ticker)

In [ ]:
app = main_app.app

In [ ]:
from myflask.sentiment import generate_image

In [ ]:
dataframe = pd.DataFrame()

In [ ]:
@app.route('/sentiment', methods=['GET', 'POST'])
def get_sentiment_page():
    if request.method=='GET':
        return render_template('sentiment.html', truth=False)
    elif request.method=='POST':
        ticker = request.form['ticker']
        generate_image(ticker)
        url = f"images/{ticker}.png"
        # Tweet Analysis
        df = pd.DataFrame()
        df = query(BEARER_TOKEN, ticker, df)
        df = refine_df(tzone, df, ticker)
        make_data(ticker, df)
        global dataframe
        dataframe = df
        url_tweet = f"images/{ticker}_tweet.png"
        return render_template('sentiment.html', truth=True, url=url, url_tweet = url_tweet)

In [ ]:
run_with_ngrok(app)
app.run()

 * Serving Flask app "myflask.main_app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://7a63-34-125-3-183.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [11/Aug/2022 08:46:20] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Aug/2022 08:46:20] "GET /news_style.css HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [11/Aug/2022 08:46:21] "GET /static/vendor/bootstrap/css/bootstrap.min.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Aug/2022 08:46:21] "GET /static/vendor/glightbox/css/glightbox.min.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Aug/2022 08:46:21] "GET /static/vendor/bootstrap-icons/bootstrap-icons.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Aug/2022 08:46:21] "GET /static/vendor/swiper/swiper-bundle.min.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Aug/2022 08:46:21] "GET /static/css/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Aug/2022 08:46:21] "GET /static/vendor/remixicon/remixicon.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Aug/2022 08:46:22] "GET /static/vendor/aos/aos.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Aug/2022 08:46:22] "GET /s

400
Range:  13.98004150390625
872.6495496812871 13.98004150390625 875.0927505493164
872.7815772949896 13.98004150390625 875.0927505493164
872.4620816715509 13.98004150390625 875.0927505493164
871.1007811363572 13.98004150390625 875.0927505493164
873.4395444310821 13.98004150390625 875.0927505493164
872.4076143745384 13.98004150390625 875.0927505493164
Mean sentiments [875.0927505493164, 875.0927505493164, 872.6495496812871, 872.7815772949896, 872.4620816715509, 871.1007811363572, 873.4395444310821, 872.4076143745384]


...

ERROR:myflask.main_app:Exception on /sentiment [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.7/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-19-22897c876db0>", line 11, in get_sentiment_page
    df = query(BEARER_TOKEN, ticker, df)
  File 

400
400


ERROR:myflask.main_app:Exception on /sentiment [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.7/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-19-22897c876db0>", line 12, in get_sentiment_page
    df = refine_df(tzone, df, ticker)
  File "<i

Range:  0.529998779296875
120.93022061134334 0.529998779296875 120.74178791046143
120.80737070020797 0.529998779296875 120.74178791046143
120.76667628535219 0.529998779296875 120.74178791046143
120.80594606976665 0.529998779296875 120.74178791046143
120.86025074922482 0.529998779296875 120.74178791046143
120.94563136027236 0.529998779296875 120.74178791046143
Mean sentiments [120.74178791046143, 120.74178791046143, 120.93022061134334, 120.80737070020797, 120.76667628535219, 120.80594606976665, 120.86025074922482, 120.94563136027236]


INFO:werkzeug:127.0.0.1 - - [11/Aug/2022 08:51:18] "POST /sentiment HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Aug/2022 08:51:18] "GET /news_style.css HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [11/Aug/2022 08:51:18] "GET /static/images/goog_tweet.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Aug/2022 08:55:28] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Aug/2022 08:55:28] "GET /news_style.css HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [11/Aug/2022 08:55:28] "GET /static/img/hero-bg.png HTTP/1.1" 404 -


# Hide this

In [ ]:
dataframe['text'][42]

'Musk sells Tesla stock worth $6.9 billion as possibility of forced Twitter deal rises '

In [ ]:
dataframe

,id,created_at,text,probability,sentiment,polarity,total,new_created_at,America/New_York Timezone
0,1557447444492800002,2022-08-10T19:23:01.000Z,$ARTL $EDTX $ROSEW $KLDO $ELMS $EVLV $LIBYW $T...,0.715107,POSITIVE,1,0.715107,2022-08-10 19:23:01,2022-08-10 15:23:01-04:00
1,1557447433180921862,2022-08-10T19:22:58.000Z,$TSLA climbing into the close?,0.994930,NEGATIVE,-1,-0.994930,2022-08-10 19:22:58,2022-08-10 15:22:58-04:00
2,1557447426147028994,2022-08-10T19:22:57.000Z,Dow Jones Pops; Tesla Gains As Elon Musk Makes...,0.539958,POSITIVE,1,0.539958,2022-08-10 19:22:57,2022-08-10 15:22:57-04:00
3,1557447416378322946,2022-08-10T19:22:54.000Z,#CWtop100 #97. Lobbyist is widely respected b...,0.998302,POSITIVE,1,0.998302,2022-08-10 19:22:54,2022-08-10 15:22:54-04:00
4,1557447409894146049,2022-08-10T19:22:53.000Z,Giga Ontario May Be Next For Tesla - CleanTech...,0.722628,NEGATIVE,-1,-0.722628,2022-08-10 19:22:53,2022-08-10 15:22:53-04:00
...,...,...,...,...,...,...,...,...,...
225,1557114982340362241,2022-08-09T21:21:56.000Z,"California Love: Tesla Model Y, Model 3 Are St...",0.996497,POSITIVE,1,0.996497,2022-08-09 21:21:56,2022-08-09 17:21:56-04:00
226,1557114849791991810,2022-08-09T21:21:24.000Z,Brampton can be a good spot for Elon Musk to s...,0.987596,POSITIVE,1,0.987596,2022-08-09 21:21:24,2022-08-09 17:21:24-04:00
227,1557114764014256129,2022-08-09T21:21:04.000Z,Tesla: Musk announces 'bad news' about Cybertr...,0.978648,POSITIVE,1,0.978648,2022-08-09 21:21:04,2022-08-09 17:21:04-04:00
228,1557114722696192006,2022-08-09T21:20:54.000Z,🇺🇸 NEW 22 MS PLAID in Black/Cream/21 with 6 Mi...,0.946472,POSITIVE,1,0.946472,2022-08-09 21:20:54,2022-08-09 17:20:54-04:00


In [ ]:
len(dataframe)

In [ ]:
first = dataframe

In [ ]:
len(dataframe)

229

In [ ]:
tweet = "RT : I\'ve got a crush on Cyber Truck and create the TORIX \"Cyber Truck X TORIX\" This video is fake But hope to see our TORIX on…"

In [ ]:
tweet == dataframe['text'][224]

True

In [ ]:
len(dataframe[tweet == dataframe['text']])

26

In [ ]:
dataframe['text'][225]

'RT : I\'ve got a crush on Cyber Truck and create the TORIX "Cyber Truck X TORIX" This video is fake But hope to see our TORIX on…'

In [ ]:
dataframe

,id,created_at,text,probability,sentiment,polarity,total,new_created_at,America/New_York Timezone
0,1498567374861225989,2022-03-01T07:54:38.000Z,New Tesla Cybertruck prototype spotted with be...,0.997586,NEGATIVE,-1,-0.997586,2022-03-01 07:54:38,2022-03-01 02:54:38-05:00
1,1498567349196230659,2022-03-01T07:54:32.000Z,Twitter's on the Rise: $TSLA Price: $870.43 Me...,0.985091,NEGATIVE,-1,-0.985091,2022-03-01 07:54:32,2022-03-01 02:54:32-05:00
2,1498567339264126977,2022-03-01T07:54:30.000Z,RT : $TSLA is +86% (+$364 per share) since thi...,0.999986,NEGATIVE,-1,-0.999986,2022-03-01 07:54:30,2022-03-01 02:54:30-05:00
3,1498567335879143425,2022-03-01T07:54:29.000Z,"RT : giveaway 200$ | 2,800,000 IDR ✅ 100$ -- r...",0.996083,NEGATIVE,-1,-0.996083,2022-03-01 07:54:29,2022-03-01 02:54:29-05:00
4,1498567329042472965,2022-03-01T07:54:27.000Z,"RT : giveaway 200$ | 2,800,000 IDR ✅ 100$ -- r...",0.996083,NEGATIVE,-1,-0.996083,2022-03-01 07:54:27,2022-03-01 02:54:27-05:00
...,...,...,...,...,...,...,...,...,...
225,1498235135912980482,2022-02-28T09:54:26.000Z,"Tesla to open Berlin gigafactory on Friday, de...",0.974727,POSITIVE,1,0.974727,2022-02-28 09:54:26,2022-02-28 04:54:26-05:00
226,1498235119273865230,2022-02-28T09:54:22.000Z,RT : $TSLA selling huge numbers of electric ve...,0.985125,NEGATIVE,-1,-0.985125,2022-02-28 09:54:22,2022-02-28 04:54:22-05:00
227,1498235118699180035,2022-02-28T09:54:22.000Z,RT : Tesla Is Hosting a Get to Know Your Tesla...,0.996085,POSITIVE,1,0.996085,2022-02-28 09:54:22,2022-02-28 04:54:22-05:00
228,1498235086738653184,2022-02-28T09:54:15.000Z,"RT : 🎉 #NFTGiveaway 🎉 Introducing 🔥 4,999 uni...",0.990320,POSITIVE,1,0.990320,2022-02-28 09:54:15,2022-02-28 04:54:15-05:00


In [ ]:
os.path.isfile('/content/gdrive/My Drive/PPL Project/news_api-main/myflask/static/data/tsla.json')

True

In [ ]:
hist = pd.read_json('/content/gdrive/My Drive/PPL Project/news_api-main/myflask/static/data/tsla.json')

In [ ]:
hist.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
2012-03-01,6.702,6.900,6.662,6.882,3517500,0,0
2012-03-02,6.880,6.900,6.742,6.808,2750000,0,0
2012-03-05,6.870,6.880,6.692,6.754,2335000,0,0
2012-03-06,6.650,6.656,6.524,6.622,2869000,0,0
2012-03-07,6.624,6.662,6.582,6.624,1824500,0,0


In [ ]:
hist.reset_index(inplace=True)

In [ ]:
hist.head()

,index,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2012-03-01,6.702,6.900,6.662,6.882,3517500,0,0
1,2012-03-02,6.880,6.900,6.742,6.808,2750000,0,0
2,2012-03-05,6.870,6.880,6.692,6.754,2335000,0,0
3,2012-03-06,6.650,6.656,6.524,6.622,2869000,0,0
4,2012-03-07,6.624,6.662,6.582,6.624,1824500,0,0


In [ ]:
ticker = 'GOOG'
stock = yf.Ticker(f"{ticker}")
company_name = stock.info

In [ ]:
company_name

{'52WeekChange': 0.29604888,
 'SandP52WeekChange': 0.13289964,
 'address1': '1600 Amphitheatre Parkway',
 'algorithm': None,
 'annualHoldingsTurnover': None,
 'annualReportExpenseRatio': None,
 'ask': 0,
 'askSize': 1200,
 'averageDailyVolume10Day': 1576060,
 'averageVolume': 1499064,
 'averageVolume10days': 1576060,
 'beta': 1.069617,
 'beta3Year': None,
 'bid': 0,
 'bidSize': 1000,
 'bookValue': 380.044,
 'category': None,
 'circulatingSupply': None,
 'city': 'Mountain View',
 'companyOfficers': [],
 'country': 'United States',
 'currency': 'USD',
 'currentPrice': 2697.82,
 'currentRatio': 2.928,
 'dateShortInterest': 1644883200,
 'dayHigh': 2711.93,
 'dayLow': 2656.505,
 'debtToEquity': 11.329,
 'dividendRate': None,
 'dividendYield': None,
 'earningsGrowth': 0.377,
 'earningsQuarterlyGrowth': 0.356,
 'ebitda': 91144003584,
 'ebitdaMargins': 0.35377,
 'enterpriseToEbitda': 18.345,
 'enterpriseToRevenue': 6.49,
 'enterpriseValue': 1672040677376,
 'exDividendDate': None,
 'exchange': 